In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import PIL
from skimage.feature import local_binary_pattern, greycomatrix, greycoprops
from skimage.filters import gabor
import numpy as np
import pickle
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train', 'test', 'train.csv', 'sample_submission.csv']


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import PIL
from skimage.feature import local_binary_pattern, greycomatrix, greycoprops
from skimage.filters import gabor
import numpy as np
import pickle
import matplotlib.pyplot as plt
class IMGFeatures:
    def __init__(self):        
        self.lbp_energy= 0
        self.lbp_entropy = 0
        self.contrast=0
        self.dissimilarity=0
        self.homogeneity=0
        self.energy =0
        self.correlation =0        
        self.g_energy=0
        self.g_entropy=0
        
    def load_image(self,path):
        self.img = PIL.Image.open(path)
        self.img_gray = self.img.convert('L') #Converting to grayscale
        self.img_arr = np.array(self.img_gray) #Converting to array
        #plt.imshow(self.img)
    def calculate_features(self):
        feat_lbp = local_binary_pattern(self.img_arr,8,1,'uniform') #Radius = 1, No. of neighbours = 8
        feat_lbp = np.uint8((feat_lbp/feat_lbp.max())*255) #Converting to unit8
        lbp_img = PIL.Image.fromarray(feat_lbp) #Conversion from array to PIL image
        
        
        lbp_hist,_ = np.histogram(feat_lbp,8)
        lbp_hist = np.array(lbp_hist,dtype=float)
        lbp_prob = np.divide(lbp_hist,np.sum(lbp_hist))
        self.lbp_energy = np.sum(lbp_prob**2)
        self.lbp_entropy = -np.sum(np.multiply(lbp_prob,np.log2(lbp_prob)))
        
        gCoMat = greycomatrix(self.img_arr, [2], [0],256,symmetric=True, normed=True) # Co-occurance matrix
        self.contrast = greycoprops(gCoMat, prop='contrast')
        self.dissimilarity = greycoprops(gCoMat, prop='dissimilarity')
        self.homogeneity = greycoprops(gCoMat, prop='homogeneity')
        self.energy = greycoprops(gCoMat, prop='energy')
        self.correlation = greycoprops(gCoMat, prop='correlation')
        self.feat_glcm= np.array([self.contrast[0][0],self.dissimilarity[0][0],self.homogeneity[0][0],self.energy[0][0],self.correlation[0][0]])
        # Gabor filter
        gaborFilt_real,gaborFilt_imag = gabor(self.img_arr,frequency=0.6)
        gaborFilt = (gaborFilt_real**2+gaborFilt_imag**2)//2
                        
        gabor_hist,_ = np.histogram(gaborFilt,8)
        gabor_hist = np.array(gabor_hist,dtype=float)
        gabor_prob = np.divide(gabor_hist,np.sum(gabor_hist))
        self.g_energy = np.sum(gabor_prob**2)
        self.g_entropy = -np.sum(np.multiply(gabor_prob,np.log2(gabor_prob)))
        self.concat_feat = np.concatenate(([self.lbp_energy,self.lbp_entropy],self.feat_glcm,[self.g_energy,self.g_entropy]),axis=0)
        

In [3]:
train_df=pd.read_csv('../input/train.csv')
print(train_df[0:5].Id)
train_df=train_df[0:5000]
print(train_df.shape)
#we treat this like unet arquitect and like a mask we replace target to 0 to 27 outputs from 0 to 1 values
NUM_CLASSES=28
y_df=[[int(i) for i in s.split()] for s in train_df['Target']]
y_df_final=np.zeros((train_df.shape[0],NUM_CLASSES)).astype(int)

for i,x in enumerate(y_df):
    for val in x:
        #print(x,val,i)    
        y_df_final[i,val]=1
print(y_df_final[1])   
print(y_df_final.shape)

0    00070df0-bbc3-11e8-b2bc-ac1f6b6435d0
1    000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0
2    000a9596-bbc4-11e8-b2bc-ac1f6b6435d0
3    000c99ba-bba4-11e8-b2b9-ac1f6b6435d0
4    001838f8-bbca-11e8-b2bc-ac1f6b6435d0
Name: Id, dtype: object
(5000, 2)
[1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(5000, 28)


In [4]:
#one demo features
IMG=IMGFeatures()
full_image_path='../input/train/'+train_df.values[0][0]+"_green.png"
IMG.load_image(full_image_path)
IMG.calculate_features()
print(IMG.concat_feat)

#calculate all features for training set and green images
label = []
featLength = 2+5+2
trainFeats = np.zeros((len(train_df),featLength)) #Feature vector of each image
for tr in range(len(train_df)):
    #print(str(tr+1)+'/'+str(len(train_df)))
    IMG=IMGFeatures()
    full_image_path='../input/train/'+train_df.values[tr][0]+"_green.png"
    IMG.load_image(full_image_path)
    IMG.calculate_features()
    trainFeats[tr,:] = IMG.concat_feat 
    
print("DONE")

/opt/conda/lib/python3.6/site-packages/skimage/feature/texture.py:109: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(image.dtype, np.float):


[3.89184524e-01 1.90975749e+00 2.29446297e+02 9.37285922e+00
 2.53350515e-01 1.83076731e-01 2.19140414e-01 9.04467326e-01
 3.64650388e-01]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in log2
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in multiply


DONE


In [5]:
#repalce some nan columns with 0
final=np.nan_to_num(trainFeats)
xpd=pd.DataFrame(final)
xpd.isnull()
print(xpd[xpd.isnull().any(axis=1)])

# Normalizing the train features to the range [0,1]
trMaxs = np.max(final,axis=0) #Finding maximum along each column
trMins = np.amin(final,axis=0) #Finding maximum along each column
trMaxs_rep = np.tile(trMaxs,(len(train_df),1)) #Repeating the maximum value along the rows
trMins_rep = np.tile(trMins,(len(train_df),1)) #Repeating the minimum value along the rows
trainFeatsNorm = np.divide(final-trMins_rep,trMaxs_rep) #Element-wise division

print(trainFeatsNorm.shape)
print(trainFeatsNorm[0])

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Index: []
(5000, 9)
[0.20197185 0.65932252 0.04818324 0.1987817  0.2261949  0.17939274
 0.18945008 0.67209592 0.14398139]


In [6]:
trainFeatsRed = np.zeros((len(train_df),featLength)) #Feature vector of each image
for tr in range(len(train_df)):
    #print(str(tr+1)+'/'+str(len(train_df)))
    IMG=IMGFeatures()
    full_image_path='../input/train/'+train_df.values[tr][0]+"_red.png"
    IMG.load_image(full_image_path)
    IMG.calculate_features()
    trainFeatsRed[tr,:] = IMG.concat_feat 
    
print("DONE")

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in log2
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in multiply


DONE


In [7]:
#repalce some nan columns values with 0
final=np.nan_to_num(trainFeatsRed)
xpd=pd.DataFrame(final)
xpd.isnull()
print(xpd[xpd.isnull().any(axis=1)])

# Normalizing the train features to the range [0,1]
trMaxs = np.max(final,axis=0) #Finding maximum along each column
trMins = np.amin(final,axis=0) #Finding maximum along each column
trMaxs_rep = np.tile(trMaxs,(len(train_df),1)) #Repeating the maximum value along the rows
trMins_rep = np.tile(trMins,(len(train_df),1)) #Repeating the minimum value along the rows
trainFeatsNormR = np.divide(final-trMins_rep,trMaxs_rep) #Element-wise division

print(trainFeatsNormR.shape)
print(trainFeatsNormR[0])

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Index: []
(5000, 9)
[0.17078801 0.69338936 0.03752456 0.16158449 0.22302551 0.18667127
 0.17738475 0.66667844 0.20679227]


In [8]:
trainFeatsBlue = np.zeros((len(train_df),featLength)) #Feature vector of each image
for tr in range(len(train_df)):
    #print(str(tr+1)+'/'+str(len(train_df)))
    IMG=IMGFeatures()
    full_image_path='../input/train/'+train_df.values[tr][0]+"_blue.png"
    IMG.load_image(full_image_path)
    IMG.calculate_features()
    trainFeatsBlue[tr,:] = IMG.concat_feat 
    
print("DONE")

#repalce some nan columns values with 0
final=np.nan_to_num(trainFeatsBlue)
xpd=pd.DataFrame(final)
xpd.isnull()
print(xpd[xpd.isnull().any(axis=1)])

# Normalizing the train features to the range [0,1]
trMaxs = np.max(final,axis=0) #Finding maximum along each column
trMins = np.amin(final,axis=0) #Finding maximum along each column
trMaxs_rep = np.tile(trMaxs,(len(train_df),1)) #Repeating the maximum value along the rows
trMins_rep = np.tile(trMins,(len(train_df),1)) #Repeating the minimum value along the rows
trainFeatsNormB = np.divide(final-trMins_rep,trMaxs_rep) #Element-wise division

print(trainFeatsNormB.shape)
print(trainFeatsNormB[0])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in log2
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in multiply
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in log2
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in multiply


DONE
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Index: []
(5000, 9)
[0.25532122 0.6610356  0.01762039 0.12401083 0.31776137 0.41269838
 0.55388748 0.5315787  0.03338543]


In [9]:
##join trainFeatsNorm and trainFeatsNormR adn trainFeatsNormB (green and red and blue)
allfeats=np.concatenate((trainFeatsNorm,trainFeatsNormR,trainFeatsNormB),axis=-1)
print(allfeats.shape) #neet to be [size,18]

(5000, 27)


In [17]:
##join trainFeatsNorm and trainFeatsNormR adn trainFeatsNormB (green and red and blue)
lista=[]
lista.append(trainFeatsNorm)
lista.append(trainFeatsNormR)
lista.append(trainFeatsNormB)
allconvfeat=np.array(lista)
allconvfeat=allconvfeat.reshape(allconvfeat.shape[1],allconvfeat.shape[2],allconvfeat.shape[0])
#allconvfeat=np.concatenate((trainFeatsNorm,trainFeatsNormR,trainFeatsNormB),axis=2)
print(allconvfeat.shape) #neet to be [size,18]

(5000, 9, 3)


In [24]:
#we have 9 features now we tran to train a simple classifcator and check how good are this features, ¿can imporve our CNN model?
import keras
from keras.models import Sequential,Model
from keras.layers import Dense,Convolution1D,Flatten,Activation,Dropout,MaxPooling2D,Conv2D,Input
from keras import backend as K
import tensorflow as tf
def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

def bce_f1_loss(y_true, y_pred):    
    return K.binary_crossentropy(y_true,y_pred)+f1_loss(y_true,y_pred)

def create_dense_baseline():    
    # create model
    model = Sequential()
    model.add(Dense(100, input_dim=allfeats.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(100, kernel_initializer='normal', activation='relu'))
    model.add(Dense(100, kernel_initializer='normal', activation='relu'))
    #model.add(Dense(NUM_CLASSES, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','binary_crossentropy',f1_loss,bce_f1_loss])
    return model

def create_conv_baseline():    
    # create model
    model = Sequential()
    model.add(Convolution1D(nb_filter=32, filter_length=1, input_shape=(allfeats.shape[1], 1)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(2048, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    #model.add(Dense(NUM_CLASSES, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','binary_crossentropy',f1_loss,bce_f1_loss])
    return model

def part1():
    
    minput=Input(shape=(allconvfeat.shape[1], allconvfeat.shape[2]))
    x=Convolution1D(nb_filter=32, filter_length=1, input_shape=(allfeats.shape[1], 1))(minput)
    x=Activation('relu')(x)
    x=Flatten()(x)
    
    return minput,x


def part2():
    minput=Input(shape=(27,))
    x=Dense(100)(minput)
    x=Activation('relu')(x)
    x=Dropout(0.5)(x)    
    x=Dense(100)(x)
    x=Activation('relu')(x)
    x=Dropout(0.5)(x)
    x=Dense(100)(x)
    x=Activation('relu')(x)
    x=Dropout(0.5)(x)
    
    return minput,x

def part3():
    minput=Input(shape=(512,512,3))
    x=Conv2D(32,(3,3))(minput)
    x=Activation('relu')(x)
    x=MaxPooling2D(pool_size=(2, 2))(x)
    x=Conv2D(32,(3,3))(x)
    x=Activation('relu')(x)
    x=MaxPooling2D(pool_size=(2, 2))(x)
    x=Conv2D(64,(3,3))(x)
    x=Activation('relu')(x)
    x=MaxPooling2D(pool_size=(2, 2))(x)
    x=Conv2D(64,(3,3))(x)
    x=Activation('relu')(x)
    x=MaxPooling2D(pool_size=(2, 2))(x)
    x=Conv2D(128,(3,3))(x)
    x=Activation('relu')(x)
    x=MaxPooling2D(pool_size=(2, 2))(x)
    x=Conv2D(128,(3,3))(x)
    x=Activation('relu')(x)
    x=MaxPooling2D(pool_size=(2, 2))(x)

    x=Flatten()(x)
    return minput,x
# the model so far outputs 3D feature maps (height, width, features)





Input1,model1=part1()
Input2,model2=part2()
Input3,model3=part3()
x = keras.layers.concatenate([ model2,model3])
# We stack a deep densely-connected network on top
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
main_output = Dense(NUM_CLASSES, activation='sigmoid', name='main_output')(x)
model = Model(inputs=[ Input2,Input3], outputs=main_output)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','binary_crossentropy',f1_loss,bce_f1_loss])
#model.add(Dense(28))
#model.add(Activation('sigmoid'))
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','binary_crossentropy',f1_loss,bce_f1_loss])
#model=create_baseline()



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(27, 1), filters=32, kernel_size=1)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 510, 510, 32) 896         input_9[0][0]                    
__________________________________________________________________________________________________
activation_25 (Activation)      (None, 510, 510, 32) 0           conv2d_13[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_13 (MaxPooling2D) (None, 255, 255, 32) 0           activation_25[0][0]              
__________________________________________________________________________________________________
conv2d_14 

In [20]:
print(allfeats.shape)
#allconvfeat=allfeats.reshape(-1,allfeats.shape[1],1)
print(allconvfeat.shape)

(5000, 27)
(5000, 9, 3)


In [41]:
from keras import backend
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,Conv2DTranspose,Convolution2D
from keras.models import Model
from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", name="Conv0", padding="same")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), name="MaxPool0", padding="same")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", name="Conv1", padding="same")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), name="MaxPool1", padding="same")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", name="Conv2", padding="same")`
/opt/conda/lib/python3.6/site-packages/ipykernel

ValueError: Input 0 is incompatible with layer Deconv0: expected axis -1 of input shape to have value 8 but got shape (None, 8, 64.0, 64.0)

In [98]:

#tenemos las features, ahora vamos a entrenar un autoencode para reducir las dimensiones

   #net = lays.conv2d(inputs, 32, [5, 5], stride=2, padding='SAME')
    #net = lays.conv2d(net, 16, [5, 5], stride=2, padding='SAME')
    #net = lays.conv2d(net, 8, [5, 5], stride=4, padding='SAME')
    # decoder
    # 2 x 2 x 8    ->  8 x 8 x 16
    # 8 x 8 x 16   ->  16 x 16 x 32
    # 16 x 16 x 32  ->  32 x 32 x 1
    #net = lays.conv2d_transpose(net, 16, [5, 5], stride=4, padding='SAME')
    #net = lays.conv2d_transpose(net, 32, [5, 5], stride=2, padding='SAME')
    #net = lays.conv2d_transpose(net, 1, [5, 5], stride=2, padding='SAME', activation_fn=tf.nn.tanh)
    
#input_img = Input(shape=(512, 512, 3))  # adapt this if using `channels_first` image data format
#x=Conv2D(32,(5,5),strides=2,padding='same',activation="relu")(input_img)
#x=Conv2D(16,(5,5),strides=2,padding='same',activation="relu")(x)
#x=Conv2D(8,(5,5),strides=4,padding='same',name="encoded",activation="relu")(x)
#x=Conv2DTranspose(16,(5,5),strides=4,padding='same',activation="relu")(x)
#x=Conv2DTranspose(32,(5,5),strides=2,padding='same',activation="relu")(x)
#x=Conv2DTranspose(3,(5,5),strides=2,padding='same',activation="sigmoid")(x)

#autoencoder = Model(input_img, x)
img_size_target=284
input_img = Input(shape=(img_size_target, img_size_target, 3))  # adapt this if using `channels_first` image data format
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.summary()

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img#,save_img
from skimage.transform import resize
from skimage import util 

def downsample(img):# not used
    
    if img_size_target==512:
        return img
    else:
        return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True)
    
def data_generator_ae(traind_df,batch_size):
    while True:        
        random_indexes = np.random.choice(len(traind_df), batch_size)        
        batch_images = np.empty((batch_size, img_size_target, img_size_target, 3))
        
        for x in range(batch_size):            
            for i, idx in enumerate(random_indexes):
              #  name="sdasd"                
                #name="../input/train/{}_green.png".format(np.squeeze(traind_df[idx]))
                name="../input/train/{}_red.png".format(traind_df.values[idx][0])
                R=(downsample(np.array(load_img(name, grayscale=True))) / 1)-0                
                name="../input/train/{}_green.png".format(traind_df.values[idx][0])
                G=(downsample(np.array(load_img(name, grayscale=True))) / 1)-0
                name="../input/train/{}_blue.png".format(traind_df.values[idx][0])
                B=(downsample(np.array(load_img(name, grayscale=True))) / 1)-0
                image=np.concatenate((R.reshape(img_size_target,img_size_target,1),G.reshape(img_size_target,img_size_target,1),B.reshape(img_size_target,img_size_target,1)),axis=2)
                batch_images[i] = util.invert((image/255))
                
        yield batch_images,batch_images
    
ret,_=next(data_generator_ae(train_df,1))   
print(ret[0].shape)                   



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_54 (InputLayer)        (None, 284, 284, 3)       0         
_________________________________________________________________
conv2d_245 (Conv2D)          (None, 284, 284, 16)      448       
_________________________________________________________________
max_pooling2d_93 (MaxPooling (None, 142, 142, 16)      0         
_________________________________________________________________
conv2d_246 (Conv2D)          (None, 142, 142, 8)       1160      
_________________________________________________________________
max_pooling2d_94 (MaxPooling (None, 71, 71, 8)         0         
_________________________________________________________________
conv2d_247 (Conv2D)          (None, 71, 71, 8)         584       
_________________________________________________________________
max_pooling2d_95 (MaxPooling (None, 36, 36, 8)         0         
__________

In [ ]:
autoencoder.fit_generator(data_generator_ae(train_df,10),steps_per_epoch=10,epochs=10,verbose=1)

In [ ]:
ret,_=next(data_generator_ae(train_df,1))   
print(ret[0].shape) 
plt.imshow((ret[0]))
plt.show()
recon=autoencoder.predict(ret)
plt.imshow((recon[0]))
plt.show()

(284, 284, 3)


In [25]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img#,save_img

def data_generator(allconvfeat,allfeats,traind_df,y_labels,batch_size):
    while True:        
        random_indexes = np.random.choice(len(traind_df), batch_size)
        batch_conv = np.empty((batch_size, allconvfeat.shape[1], allconvfeat.shape[2]))
        batch_dense = np.empty((batch_size, 27))
        batch_images = np.empty((batch_size, 512, 512, 3))
        batch_labels = np.zeros((batch_size, 28))
        for x in range(batch_size):            
            for i, idx in enumerate(random_indexes):
              #  name="sdasd"                
                #name="../input/train/{}_green.png".format(np.squeeze(traind_df[idx]))
                name="../input/train/{}_red.png".format(traind_df.values[idx][0])
                R=np.array(load_img(name, grayscale=True)) / 255                
                name="../input/train/{}_green.png".format(traind_df.values[idx][0])
                G=np.array(load_img(name, grayscale=True)) / 255
                name="../input/train/{}_blue.png".format(traind_df.values[idx][0])
                B=np.array(load_img(name, grayscale=True)) / 255
                image=np.concatenate((R.reshape(512,512,1),G.reshape(512,512,1),B.reshape(512,512,1)),axis=2)
                batch_images[i] = image
                batch_labels[i]=y_labels[idx]
                batch_conv[i]=allconvfeat[idx]
                batch_dense[i]=allfeats[idx]
        yield [batch_dense,batch_images],batch_labels
    
ret,_=next(data_generator(allconvfeat,allfeats,train_df,y_df_final,1))   
print(ret[0].shape)                   

(1, 27)


In [26]:
model.fit_generator(data_generator(allconvfeat,allfeats,train_df,y_df_final,10),steps_per_epoch=50,epochs=10,verbose=1)

Epoch 1/10
50/50 [==============================] - 83s 2s/step - loss: 0.2417 - acc: 0.9288 - binary_crossentropy: 0.2417 - f1_loss: 0.9551 - bce_f1_loss: 1.1967
Epoch 2/10
50/50 [==============================] - 81s 2s/step - loss: 0.1816 - acc: 0.9409 - binary_crossentropy: 0.1816 - f1_loss: 0.9549 - bce_f1_loss: 1.1365
Epoch 3/10
50/50 [==============================] - 82s 2s/step - loss: 0.1820 - acc: 0.9421 - binary_crossentropy: 0.1820 - f1_loss: 0.9576 - bce_f1_loss: 1.1395
Epoch 4/10
50/50 [==============================] - 82s 2s/step - loss: 0.1812 - acc: 0.9409 - binary_crossentropy: 0.1812 - f1_loss: 0.9567 - bce_f1_loss: 1.1379
Epoch 5/10
50/50 [==============================] - 81s 2s/step - loss: 0.1805 - acc: 0.9405 - binary_crossentropy: 0.1805 - f1_loss: 0.9558 - bce_f1_loss: 1.1362
Epoch 6/10
50/50 [==============================] - 81s 2s/step - loss: 0.1786 - acc: 0.9394 - binary_crossentropy: 0.1786 - f1_loss: 0.9548 - bce_f1_loss: 1.1333
Epoch 7/10
50/50 [====

In [ ]:
#model.fit([allconvfeat,allfeats],y_df_final,batch_size=320,epochs=300,validation_split=0.1,verbose=1)

In [ ]:
model.fit(allconvfeat,y_df_final,batch_size=320,epochs=300,validation_split=0.1,verbose=1)